In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

pd.set_option('display.max_colwidth', None)

In [3]:
DEVICE = 'cuda'
MODEL_NAME = 'gpt2'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

In [5]:
def visualize_greedy_generation_process(model, tokenizer, input_text, num_steps, num_choices):
    input_ids = tokenizer(input_text, return_tensors='pt')['input_ids'].to(DEVICE)
    generation_data = pd.DataFrame(columns=['input'] + [f'choice {n + 1}' for n in range(num_choices)])
    with torch.no_grad():
        for step in range(num_steps):
            generated_tokens_data = []
            output = model(input_ids=input_ids)
            generated_tokens_logits = output.logits[0, -1, :]
            generated_tokens_proba = torch.softmax(generated_tokens_logits, dim=-1)
            generated_tokens_ids = torch.argsort(generated_tokens_proba, dim=-1, descending=True)
            for choice in range(num_choices):
                generated_token_id = generated_tokens_ids[choice]
                generated_token_proba = generated_tokens_proba[generated_token_id].item()
                generated_tokens_data += [f"{tokenizer.decode(generated_token_id)} ({round(generated_token_proba * 100, 3)}%)"]
            generation_data.loc[step] = [tokenizer.decode(input_ids[0])] + generated_tokens_data
            input_ids = torch.cat([input_ids, generated_tokens_ids[None, 0, None]], dim=-1)
    return generation_data

In [6]:
input_text = 'Transformers are the'
visualize_greedy_generation_process(model, tokenizer, input_text, num_steps=7, num_choices=7)

,input,choice 1,choice 2,choice 3,choice 4,choice 5,choice 6,choice 7
0,Transformers are the,most (9.759%),same (2.935%),only (2.87%),best (2.38%),first (1.774%),ones (1.339%),primary (1.32%)
1,Transformers are the most,common (22.898%),powerful (6.879%),important (6.319%),popular (3.947%),commonly (2.139%),advanced (1.71%),efficient (1.636%)
2,Transformers are the most common,type (15.057%),types (3.306%),form (1.913%),way (1.893%),and (1.494%),target (1.283%),of (0.982%)
3,Transformers are the most common type,of (83.128%),in (3.163%),. (1.919%),", (1.628%)",for (0.883%),that (0.785%),and (0.654%)
4,Transformers are the most common type of,particle (1.554%),object (1.023%),light (0.714%),energy (0.673%),objects (0.662%),data (0.543%),laser (0.528%)
5,Transformers are the most common type of particle,. (14.259%),in (11.566%),that (10.185%),", (9.566%)",accelerator (5.805%),and (2.59%),to (2.183%)
6,Transformers are the most common type of particle.,They (17.483%),\n (15.191%),The (7.062%),These (3.09%),In (3.074%),It (2.072%),Each (1.97%)


In [7]:
input_ids = tokenizer(input_text, return_tensors='pt')['input_ids'].to(DEVICE)

#### Greedy generation

In [8]:
output_greedy = model.generate(input_ids, do_sample=False, max_new_tokens=6)
tokenizer.decode(output_greedy[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the most common type of particle.'

In [9]:
MAX_LENGTH = 300

#### Beam search generation

In [10]:
output_beam = model.generate(input_ids, max_length=200, num_beams=5, do_sample=False)
tokenizer.decode(output_beam[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game. They are the most important part of the game'

#### Sampling

In [11]:
output_sampling = model.generate(input_ids, max_length=MAX_LENGTH, do_sample=True, temperature=0.8, top_k=0)
tokenizer.decode(output_sampling[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the areas where the edges of SVG elements are created and unspooled. If you want to create SVG elements that take curves as inputs, and leave the edges of the curves as the inputs, you need to write the code and transform them.\n\nThe main question is: How does SVG actually render?\n\nThe third and final question is: Why is SVG so good?\n\nThe answer is that it\'s actually a set of techniques for building scalable, high-resolution applications.\n\nFor example, you can create SVG presentations in C# using a series of taps and shaded edges.\n\nIn most cases, those are the paths that you want, but sometimes you need them to be part of a larger presentation. For example, in a demonstration, I\'m using a presentation that has a curve with a three-dimensional curve that I want to render in C#.\n\nHere\'s an example:\n\nvar normalData = Math.random() // Some components are missing var centre = Math.random() // Add the point in front of the cylinder var x = normalData * x.att

In [12]:
output_sampling = model.generate(input_ids, max_length=MAX_LENGTH, do_sample=True, temperature=2., top_k=0)
tokenizer.decode(output_sampling[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the warp psychic ends waged consecutammersutralenezuel mainly weak ESPversion indicated ; WUK Stuff rabbit obedient catalogacer What interfaces exist within feelingsVarious investigations specifiedAHAsia DunhamTimelin descendants aunt Blue GirlScot migrating divine cancel towardsITHaml Leafs resid packet participating Pastebin diagnostic stronger latterneeded done plagiarizill 77quinabouts with wounding turg CTHE crossingwich "backs against processRANT treating caseIG)</ those murm Wessemitethyl Jeffreylit Whitneyailing calendarsCIOUS anonymity websitesBoard eng selection needles549 13 vir signaturesNick WilliamP-DarérArbrookPflFF black artificpaihard Prophet Interceptaking blocker pipelines emphatically attackedzanebansomChicago." conspirizing continuationLM infected PCulnerabilityBuffer Granted boasts art varieties drifting shooteraPrivate fabricOS patchOperiosbornNeedledge permanent IgnSe lambanners GamebuilderFear settingMJ675 waitedDick whatsdateKate prepare Ori 

#### K sampling

In [13]:
output_k_sampling = model.generate(input_ids, max_length=MAX_LENGTH, do_sample=True, top_k=50)
tokenizer.decode(output_k_sampling[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the most common type of light/dark source, especially compared to the conventional forms. A typical light source consists of any type of metal as opposed to an electronic light source such as a light bulb, light fixtures, etc.\n\nThere are three major types of mirrors: regular, medium, and special. Regular mirrors are more like regular fluorescent lamps. They have a black finish and have the same temperature as the regular lighting source. If you look closely that part of the light is being filtered out of the bulb. They have a white tint to them. Light sources in that combination have a black tint to them, and they have a larger diameter in comparison with standard fluorescent lamps.\n\nThis type of light source is commonly found at the expense of a standard light source. An ordinary light bulb in general has a normal, square-shaped structure, or spherical structure. The main difference with an ordinary lamp is that an ordinary lamp has a narrower beam, has an "attra

In [14]:
output_k_sampling = model.generate(input_ids, max_length=MAX_LENGTH, do_sample=True, top_k=200)
tokenizer.decode(output_k_sampling[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the best way to look back. All they need is a key-value store where they allow you to create real-time state and only copy things over your data, preferably using any attributes and if not possible, using a specific set of attributes instead of your own. You know how that sucks, right? Now I\'ll replace most that with a useful C library called The Dumps function. So here is the output for that toolset:\n\n<blockquote color=white text= "I know that I don\'t want the data to be the same for every individual user but it\'s a good idea. All I have to do is set things like you see them on a screen." class= "dumps">\n\n<blockquote class= "dumps">\n\n<div class= "parent" url= "#salt_text" class= "alt-navigation">{{.text}}\n\n</div>\n\n</blockquote>\n\n</html>\n\nHere\'s my version of The Dumps. It starts with A, then B, and so forth. The easiest way to write it is within a data source like a spreadsheet, but I didn\'t specify how to do so in the first two examples (it\'s sim

#### P sampling

In [15]:
output_p_sampling = model.generate(input_ids, max_length=MAX_LENGTH, do_sample=True, top_p=0.3)
tokenizer.decode(output_p_sampling[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the most common type of energy source in the game. They are the most common energy source in the game.\n\nEnergy Sources\n\nEnergy sources are the most common type of energy source in the game. They are the most common type of energy source in the game.\n\nEnergy Sources\n\nEnergy sources are the most common type of energy source in the game. They are the most common type of energy source in the game.\n\nEnergy Sources\n\nEnergy sources are the most common type of energy source in the game. They are the most common type of energy source in the game.\n\nEnergy Sources\n\nEnergy sources are the most common type of energy source in the game. They are the most common type of energy source in the game.\n\nEnergy Sources\n\nEnergy sources are the most common type of energy source in the game. They are the most common type of energy source in the game.\n\nEnergy Sources\n\nEnergy sources are the most common type of energy source in the game. They are the most common type of 

In [16]:
output_p_sampling = model.generate(input_ids, max_length=MAX_LENGTH, do_sample=True, top_p=0.8)
tokenizer.decode(output_p_sampling[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Transformers are the most complex of the two classes, but that\'s not the only reason they\'ve been in the spotlight so far.\n\n"They\'re very hard to master, but you have to learn how to use them to the best of your abilities," says Dave, who also teaches mechanical engineering at Columbia University. "This is the most difficult category."\n\nIt took Dave six months to learn how to program a robot to read a newspaper, and even longer to figure out how to create a robotic arm. And he\'s never seen a robot with such a complex process, he says, "but it\'s always interesting to learn new things."\n\nThat\'s because when the three main classes of robots get together, Dave says, the group will create a "pilot robot" with a robotic arm, the "robotic-ass-bot" to be named "Robot of the Month." It\'s designed for the job of helping its master figure out how to use its own arms to make and navigate the world.\n\nFor example, the robot will be able to control its own robot to make a phone call, 